# Final Project Decision Analysis



In the code Backpropagation 1 there is an example of a [4,5,2] network used to predict
the outcome of the variable `survived` from the features `pclass`,
`sex`,`age`,`sibsp` in the dataset "Titanic Dataset.csv"


In [30]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [31]:
titanic = pd.read_csv('Titanic Dataset.csv')
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [32]:
titanic.shape

(1309, 14)

In [33]:
titanic.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object

In [34]:
titanic.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

There are some missing values, so let's drop those observations. But, before, let's focus on the variables we want: `pclass`, `sex`,`age`,`sibsp`, and the target `survived`.

In [35]:
titanic_v = titanic[['pclass', 'sex', 'age', 'sibsp', 'survived']]
titanic_v.head()

,pclass,sex,age,sibsp,survived
0,1,female,29.00,0,1
1,1,male,0.92,1,1
2,1,female,2.00,1,0
3,1,male,30.00,1,0
4,1,female,25.00,1,0


In [36]:
titanic_v.isnull().sum()

pclass        0
sex           0
age         263
sibsp         0
survived      0
dtype: int64

In [37]:
titanic_v = titanic_v.dropna()
titanic_v.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1046 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1046 non-null   int64  
 1   sex       1046 non-null   object 
 2   age       1046 non-null   float64
 3   sibsp     1046 non-null   int64  
 4   survived  1046 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 49.0+ KB


Missing values have been correctly removed. (1309 original - 263 missing = 1046 left)

Now, we have to encode `sex`

In [38]:
dummies = pd.get_dummies(titanic_v['sex']).applymap(int) 
dummies.head(6) 

C:\Users\UX325\AppData\Local\Temp\ipykernel_10660\608239367.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dummies = pd.get_dummies(titanic_v['sex']).applymap(int)


,female,male
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0
5,0,1


In [39]:
titanic_v = pd.concat([titanic_v, dummies.female], axis=1)
titanic_v.head(2)

,pclass,sex,age,sibsp,survived,female
0,1,female,29.00,0,1,1
1,1,male,0.92,1,1,0


Now, let's define the features and target of the neural network.

In [40]:

features = ['pclass', 'female','age','sibsp']
target = 'survived'
Y = titanic_v[target].to_numpy(dtype = int )

In [41]:
X = titanic_v[features].to_numpy()

In [42]:
X

array([[ 1.  ,  1.  , 29.  ,  0.  ],
       [ 1.  ,  0.  ,  0.92,  1.  ],
       [ 1.  ,  1.  ,  2.  ,  1.  ],
       ...,
       [ 3.  ,  0.  , 26.5 ,  0.  ],
       [ 3.  ,  0.  , 27.  ,  0.  ],
       [ 3.  ,  0.  , 29.  ,  0.  ]], shape=(1046, 4))

In [43]:
Y[0:3]

array([1, 1, 0])

In [44]:
def vectorized_result(j):
    """Return a 2-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert survival 0/1 into a 
    corresponding desired output from the neural
    network. 0 -> [1,0] and 1 -> [0,1]"""
    e = np.zeros((2, 1))
    e[int(j)] = 1.0
    return e

Legend:
- Not Survived: `[1,0]`
- Survived: `[0,1]`

In [45]:
training_results = [vectorized_result(y) for y in Y]

In [46]:
training_results[0:3]

[array([[0.],
        [1.]]),
 array([[0.],
        [1.]]),
 array([[1.],
        [0.]])]

In [47]:
# Convert list of (2, 1) arrays into a single (n_samples, 2) array
training_results_array = np.hstack(training_results).T
training_results_array

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], shape=(1046, 2))

Normalize the features to improve learning efficiency and stability during training.

In [48]:
def normalise(X):
    k = X.shape[0]
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X, axis =0, ddof=1) 
    X = (X-X_mean)/X_std 
    return X, X_mean, X_std

In [49]:
X, X_mean, X_std =  normalise(X)
X[0:3]

array([[-1.43489222,  1.30163551, -0.06113283, -0.5512893 ],
       [-1.43489222, -0.76752975, -2.00930734,  0.54500083],
       [-1.43489222,  1.30163551, -1.93437755,  0.54500083]])

All under the same scale


In [50]:
training_inputs = [np.reshape(x, (4, 1)) for x in X]
training_inputs[0]

array([[-1.43489222],
       [ 1.30163551],
       [-0.06113283],
       [-0.5512893 ]])

Now, let's code the neural network architechture: Dense layer class, ReLu Activation, Softmax Activation and Categorical Cross-Entropy loss.

**Why Softmax activation + Categoriccal Cross-Entropy loss for the outputs, instead of Sigmoid + Binary Cross-Entropy?**

- Softmax ensures the outputs are mutually exclusive probabilities that sum to 1, which is perfect for one-hot encoded targets like [1, 0] or [0, 1], that we are using for the `survived` class.

- Categorical Cross-Entropy directly compares the full probability distribution to the one-hot label, penalizing confidence in the wrong class.

- Sigmoid treats each output independently, so it doesn't assume only one class is correct.

- Sigmoid is fine for multi-label problems (multiple classes can be "on"/correct), but not ideal when only one class is correct, like in this case.

Wrapping up, for our current setup with 2 output nodes, we should use softmax. If we switched the setup to 1 output node, we might have used sigmoid and adjusted the labels and loss accordingly.


In [51]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)


In [52]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


In [53]:
class Activation_Softmax:
 def forward(self, inputs):
  exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
  probabilities = exp_values / np.sum(exp_values, axis=1,keepdims=True)
  self.output = probabilities

Softmax doesn't have the backward pass as it is very complex to get the partial derivative of softmax's outputs with respect to its inputs. 

Instead, we will use a combined method in the `Activation_Softmax_Loss_CategoricalCrossentropy` class, where we will directly get the result of the partial derivative of loss with respect to softmax's inputs.

In [54]:
class Loss:
 def calculate(self, output, y):
  sample_losses = self.forward(output, y)
  data_loss = np.mean(sample_losses)
  return data_loss

In [55]:
class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values - only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples

In [56]:
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Create activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        self.output = self.activation.output
        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
            
        self.dinputs = dvalues.copy() # Copy so we can safely modify
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples

In [57]:
class Optimizer_GD:
 def __init__(self, learning_rate=0.05):
  self.learning_rate = learning_rate
 # Update parameters - follow the formula (new_weights = current_weights - learning_rate * gradient)
 def update_params(self, layer):
  layer.weights += -self.learning_rate * layer.dweights
  layer.biases += -self.learning_rate * layer.dbiases

Now that we have created all the classes, let's run the neural network with forward and backward passes.

In [58]:
np.random.seed(42) # for reproducibility
dense1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD()


for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, training_results_array)
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(training_results_array.shape) == 2:
        y_train = np.argmax(training_results_array, axis=1)
    else:
        y_train = training_results_array.copy()
    accuracy = np.mean(predictions == y_train)
    
    if not epoch % 100: # Print every 100 epochs (epoch = one complete pass through the entire training dataset)
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    
    optimizer.update_params(dense1) 
    optimizer.update_params(dense2) 


epoch: 0, acc: 0.358, loss: 0.693
epoch: 100, acc: 0.592, loss: 0.676
epoch: 200, acc: 0.592, loss: 0.671
epoch: 300, acc: 0.784, loss: 0.602
epoch: 400, acc: 0.783, loss: 0.481
epoch: 500, acc: 0.789, loss: 0.455
epoch: 600, acc: 0.792, loss: 0.449
epoch: 700, acc: 0.795, loss: 0.447
epoch: 800, acc: 0.795, loss: 0.445
epoch: 900, acc: 0.795, loss: 0.444
epoch: 1000, acc: 0.795, loss: 0.444
epoch: 1100, acc: 0.796, loss: 0.443
epoch: 1200, acc: 0.796, loss: 0.443
epoch: 1300, acc: 0.796, loss: 0.442
epoch: 1400, acc: 0.794, loss: 0.442
epoch: 1500, acc: 0.794, loss: 0.442
epoch: 1600, acc: 0.794, loss: 0.441
epoch: 1700, acc: 0.795, loss: 0.441
epoch: 1800, acc: 0.793, loss: 0.441
epoch: 1900, acc: 0.793, loss: 0.440
epoch: 2000, acc: 0.793, loss: 0.440
epoch: 2100, acc: 0.793, loss: 0.439
epoch: 2200, acc: 0.798, loss: 0.439
epoch: 2300, acc: 0.799, loss: 0.439
epoch: 2400, acc: 0.799, loss: 0.438
epoch: 2500, acc: 0.800, loss: 0.438
epoch: 2600, acc: 0.799, loss: 0.437
epoch: 2700, 

We are looking for parameters (weights and biases) that minimize loss. However, for evaluating the model's performance, we should consider the trade-off between accuracy and loss (best accuracy for lowest loss possible).

Therefore, we can appreciate that at epoch (which means, one complete pass through the entire training dataset by the learning algorithm) 6000, the loss begins to decay very very slowly, with few improvement. A bit after, at epoch 9000 we reach the peak in accuracy (0.815) with a loss of 0.450, which is almost identical to the loss of 0.444 at epoch 10000. For that reason, the parameters that we will keep for the model will be thos that gave the highest accuracy, as loss is very similar for the last 4000 iterations.

Note:
- By manually changing the learning rate, we found out 0.05 to be the best one. We started with a high learning rate to make sure we didn't got stuck in a local minima, regardless of overshooting, and gradually descreased it to reach highest accuracy and minimum loss.
    - I'm aware that we ca build a learning rate decay model to automate the gradual reduction in learning rate as we approach the minimum loss, but I think this simple approach is quite good too. 

<br>

Once we have trained and optimized parameters, we can save them and just load them again whevener we want to use the model. Just the forward pass will be required in this case:

In [59]:
# We save the weights and biases that give the best accuracy
np.save('dense1_weights.npy', dense1.weights)
np.save('dense1_biases.npy', dense1.biases)
np.save('dense2_weights.npy', dense2.weights)
np.save('dense2_biases.npy', dense2.biases)

In [60]:
dense1 = Layer_Dense(4, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Load weights and biases
dense1.weights = np.load('dense1_weights.npy')
dense1.biases = np.load('dense1_biases.npy')
dense2.weights = np.load('dense2_weights.npy')
dense2.biases = np.load('dense2_biases.npy')


# Just do a forward pass through the trained model
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss_activation.forward(dense2.output, training_results_array)

predictions = np.argmax(loss_activation.output, axis=1)
true_labels = (
    training_results_array
    if training_results_array.ndim == 1
    else np.argmax(training_results_array, axis=1)
)
accuracy = np.mean(predictions == true_labels)

print(f'Accuracy: {accuracy:.3f}')


Accuracy: 0.809


<br>

Here, you can check what have been our outputs (probability distribution between the 2 possible classes), and the final prediction made. 
- 0 = not survived
- 1 = survived

In [61]:
print('Probabilities:', loss_activation.output[:5])

Probabilities: [[0.01366455 0.98633545]
 [0.13516484 0.86483516]
 [0.01015673 0.98984327]
 [0.63633683 0.36366317]
 [0.0146259  0.9853741 ]]


In [62]:
print('Predictions:', predictions[:5])

Predictions: [1 1 1 0 1]


<br>

**a. How well is this network performing compared to logistic regression? (3p)**

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# all the data cleaning and preprocessing steps have been done prior to neural network. We are
# going to use the same data for the logistic regression model.

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = preprocessing.StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train


array([[-1.4342359 ,  1.31606464,  0.63349426,  0.53528339],
       [-0.24826042, -0.7598411 , -0.32341088, -0.54302242],
       [-1.4342359 ,  1.31606464,  0.22339205,  0.53528339],
       ...,
       [-1.4342359 ,  1.31606464, -0.73351308, -0.54302242],
       [ 0.93771507, -0.7598411 , -0.18671015, -0.54302242],
       [ 0.93771507, -0.7598411 ,  0.22339205, -0.54302242]],
      shape=(836, 4))

In [64]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [65]:
y_pred = log_reg.predict(X_test) 
y_pred_prob = log_reg.predict_proba(X_test)[:, 1]  
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, y_pred))

Logistic Regression Accuracy: 0.7238095238095238


**Answer:**

On the Titanic dataset, the neural network performs noticeably better at predicting passenger survival than logistic regression. The accuracy of the neural network was significantly higher at 0.815 than that of logistic regression, which was only about 0.724. 

This discrepancy demonstrates how the neural network's hidden layer and application of activation functions like ReLU enable it to identify complex, nonlinear relationships in the data. The linear nature of logistic regression, on the other hand, limits its capacity to simulate the relationships between characteristics like gender, age, and class. 

Despite the higher computational complexity, the neural network's results to better learn patterns or connections in data, resulting in more accurate predictions.

Basically:

- The neural network provides a clear performance improvement over logistic regression by capturing non-linear patterns in the data.



<br>


**b. Adapt the code so that you can use 5 input variables and add one variable of your choice. Did you perform better? What is the best layout of the network? (5p)**


As fifth input, I'm going to add `fare`, which is the money paid for a journey on public transport. This might be related to the survival chances of the passenger, as those who paid more, most likely have preference to be saved.

In [66]:
titanic_v = titanic[['pclass', 'sex', 'age', 'sibsp', 'fare', 'survived']]
titanic_v.isnull().sum()

pclass        0
sex           0
age         263
sibsp         0
fare          1
survived      0
dtype: int64

In [67]:
titanic_v = titanic_v.dropna()
titanic_v.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1045 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1045 non-null   int64  
 1   sex       1045 non-null   object 
 2   age       1045 non-null   float64
 3   sibsp     1045 non-null   int64  
 4   fare      1045 non-null   float64
 5   survived  1045 non-null   int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 57.1+ KB


In [68]:
dummies = pd.get_dummies(titanic_v['sex']).applymap(int)
titanic_v = pd.concat([titanic_v, dummies.female], axis=1)

features = ['pclass', 'female', 'age', 'sibsp', 'fare']
target = 'survived'


C:\Users\UX325\AppData\Local\Temp\ipykernel_10660\3209907120.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dummies = pd.get_dummies(titanic_v['sex']).applymap(int)


In [69]:
X = titanic_v[features].to_numpy()
Y = titanic_v[target].to_numpy(dtype=int)

#Normalize
X, X_mean, X_std = normalise(X)

#Re-encode Y to one-hot
training_results = [vectorized_result(y) for y in Y]
training_results_array = np.hstack(training_results).T


For a 5 input structure, we just have to change the first Layer_Dense instance (`dense1`), from (4 inputs, 5 neurons) to (5 inputs, 5 neurons)

In [70]:
np.random.seed(42) # for reproducibility
dense1 = Layer_Dense(5, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD()


for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, training_results_array)
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(training_results_array.shape) == 2:
        y_train = np.argmax(training_results_array, axis=1)
    else:
        y_train = training_results_array.copy()
    accuracy = np.mean(predictions == y_train)
    
    if not epoch % 100: 
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    
    optimizer.update_params(dense1) 
    optimizer.update_params(dense2) 


epoch: 0, acc: 0.632, loss: 0.693
epoch: 100, acc: 0.591, loss: 0.676
epoch: 200, acc: 0.591, loss: 0.659
epoch: 300, acc: 0.781, loss: 0.536
epoch: 400, acc: 0.777, loss: 0.468
epoch: 500, acc: 0.789, loss: 0.454
epoch: 600, acc: 0.796, loss: 0.449
epoch: 700, acc: 0.796, loss: 0.447
epoch: 800, acc: 0.795, loss: 0.445
epoch: 900, acc: 0.795, loss: 0.445
epoch: 1000, acc: 0.794, loss: 0.444
epoch: 1100, acc: 0.797, loss: 0.443
epoch: 1200, acc: 0.798, loss: 0.443
epoch: 1300, acc: 0.798, loss: 0.442
epoch: 1400, acc: 0.798, loss: 0.442
epoch: 1500, acc: 0.798, loss: 0.441
epoch: 1600, acc: 0.798, loss: 0.441
epoch: 1700, acc: 0.797, loss: 0.440
epoch: 1800, acc: 0.797, loss: 0.440
epoch: 1900, acc: 0.797, loss: 0.439
epoch: 2000, acc: 0.797, loss: 0.439
epoch: 2100, acc: 0.798, loss: 0.439
epoch: 2200, acc: 0.799, loss: 0.438
epoch: 2300, acc: 0.802, loss: 0.438
epoch: 2400, acc: 0.801, loss: 0.438
epoch: 2500, acc: 0.801, loss: 0.437
epoch: 2600, acc: 0.802, loss: 0.437
epoch: 2700, 

In [71]:
# We save the weights and biases that give the best accuracy
np.save('dense1_weights.npy', dense1.weights)
np.save('dense1_biases.npy', dense1.biases)
np.save('dense2_weights.npy', dense2.weights)
np.save('dense2_biases.npy', dense2.biases)

dense1 = Layer_Dense(5, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Load weights and biases
dense1.weights = np.load('dense1_weights.npy')
dense1.biases = np.load('dense1_biases.npy')
dense2.weights = np.load('dense2_weights.npy')
dense2.biases = np.load('dense2_biases.npy')


# Just do a forward pass through the trained model
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss_activation.forward(dense2.output, training_results_array)

predictions = np.argmax(loss_activation.output, axis=1)
true_labels = (
    training_results_array
    if training_results_array.ndim == 1
    else np.argmax(training_results_array, axis=1)
)
accuracy = np.mean(predictions == true_labels)

print(f'Accuracy: {accuracy:.3f}')
print('Loss:', loss)


Accuracy: 0.819
Loss: 0.42096326912466286


With a 5 input setup, using the same layout as before of 5 neurons in the hidden layer and 2 outputs, and keeping the same learning rate of 0.05, we get both better accuracy and lower loss, which means that including `fare` as variable in the model, has helped us in making more accurate predictions on Titanic survival.

However, for a better neural network layout, let's experiment with different layouts. I could keep trying many layouts for a long time, but I'm just going to try 2 modifications:
- More neurons in the hidden layer (10 neurons -- [5,10,2])
- Add another hidden layer ([5,8,4,2])

In [72]:
np.random.seed(42) # for reproducibility
dense1 = Layer_Dense(5, 10)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(10, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD()


for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, training_results_array)
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(training_results_array.shape) == 2:
        y_train = np.argmax(training_results_array, axis=1)
    else:
        y_train = training_results_array.copy()
    accuracy = np.mean(predictions == y_train)
    
    if not epoch % 100: 
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    
    optimizer.update_params(dense1) 
    optimizer.update_params(dense2) 


epoch: 0, acc: 0.448, loss: 0.693
epoch: 100, acc: 0.591, loss: 0.676
epoch: 200, acc: 0.591, loss: 0.665
epoch: 300, acc: 0.720, loss: 0.584
epoch: 400, acc: 0.777, loss: 0.479
epoch: 500, acc: 0.784, loss: 0.457
epoch: 600, acc: 0.796, loss: 0.449
epoch: 700, acc: 0.799, loss: 0.445
epoch: 800, acc: 0.801, loss: 0.443
epoch: 900, acc: 0.801, loss: 0.441
epoch: 1000, acc: 0.800, loss: 0.439
epoch: 1100, acc: 0.805, loss: 0.438
epoch: 1200, acc: 0.803, loss: 0.437
epoch: 1300, acc: 0.805, loss: 0.436
epoch: 1400, acc: 0.804, loss: 0.435
epoch: 1500, acc: 0.806, loss: 0.434
epoch: 1600, acc: 0.805, loss: 0.434
epoch: 1700, acc: 0.806, loss: 0.433
epoch: 1800, acc: 0.807, loss: 0.433
epoch: 1900, acc: 0.808, loss: 0.432
epoch: 2000, acc: 0.808, loss: 0.432
epoch: 2100, acc: 0.810, loss: 0.432
epoch: 2200, acc: 0.810, loss: 0.431
epoch: 2300, acc: 0.811, loss: 0.431
epoch: 2400, acc: 0.811, loss: 0.431
epoch: 2500, acc: 0.811, loss: 0.431
epoch: 2600, acc: 0.812, loss: 0.431
epoch: 2700, 

Even though we haven't achieved higher accuracy or significantly lower loss, we can see that with more neurons in the hidden layer, we've reached a very also high accuracy and low loss in very few iterations. 

This could be due to the fact taht a wider hidden layer can speed up learning by allowing the network to capture patterns more efficiently. However, we've learnt that for small or simple datasets, more complexity doesn’t always mean better final accuracy, it might just get you there faster.

In [73]:
np.random.seed(42) # for reproducibility
dense1 = Layer_Dense(5, 8)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(8, 4)
activation2 = Activation_ReLU()
dense3 = Layer_Dense(4, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD()


for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)
    loss = loss_activation.forward(dense3.output, training_results_array)
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(training_results_array.shape) == 2:
        y_train = np.argmax(training_results_array, axis=1)
    else:
        y_train = training_results_array.copy()
    accuracy = np.mean(predictions == y_train)
    
    if not epoch % 100: 
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}')
    
    
    loss_activation.backward(loss_activation.output, y_train)
    dense3.backward(loss_activation.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    
    optimizer.update_params(dense1) 
    optimizer.update_params(dense2) 
    optimizer.update_params(dense3)


epoch: 0, acc: 0.331, loss: 0.693
epoch: 100, acc: 0.591, loss: 0.676
epoch: 200, acc: 0.591, loss: 0.676
epoch: 300, acc: 0.591, loss: 0.676
epoch: 400, acc: 0.591, loss: 0.676
epoch: 500, acc: 0.591, loss: 0.676
epoch: 600, acc: 0.591, loss: 0.676
epoch: 700, acc: 0.591, loss: 0.676
epoch: 800, acc: 0.591, loss: 0.676
epoch: 900, acc: 0.591, loss: 0.676
epoch: 1000, acc: 0.591, loss: 0.676
epoch: 1100, acc: 0.591, loss: 0.676
epoch: 1200, acc: 0.591, loss: 0.676
epoch: 1300, acc: 0.591, loss: 0.676
epoch: 1400, acc: 0.591, loss: 0.676
epoch: 1500, acc: 0.591, loss: 0.676
epoch: 1600, acc: 0.591, loss: 0.676
epoch: 1700, acc: 0.591, loss: 0.676
epoch: 1800, acc: 0.591, loss: 0.676
epoch: 1900, acc: 0.591, loss: 0.676
epoch: 2000, acc: 0.591, loss: 0.676
epoch: 2100, acc: 0.591, loss: 0.676
epoch: 2200, acc: 0.591, loss: 0.676
epoch: 2300, acc: 0.591, loss: 0.676
epoch: 2400, acc: 0.591, loss: 0.676
epoch: 2500, acc: 0.591, loss: 0.676
epoch: 2600, acc: 0.591, loss: 0.676
epoch: 2700, 

Based on the results provided, the [5, 5, 2] layout consistently reaches high accuracy (~81.9%) and low loss (~0.421) much faster than the [5, 8, 4, 2] layout, which only reaches comparable accuracy after 5,000 epochs.

However, with the added layer, we reach higher accuracy and even lower loss (although there isn't much difference)

Given its quicker convergence, cheaper training cost, and nearly identical performance to the more intricate [5, 8, 4, 2] model, the [5, 5, 2] layout is probably the better option overall. 

The performance difference between the two is negligible, but the deeper (extra layer) layout might be taken into consideration if training efficiency is less of an issue and maximum accuracy is the primary goal. Therefore, in the majority of situations, the [5,5,2] network is the most efficient choice due to its efficiency and simplicity of model.

**Conclusion:**

> Optimizing for speed and simplicity, the [5, 5, 2] layout is clearly the most efficient and practical choice.

<br>


**c. Why do we have two output nodes? What happens if you just use one? Can you adapt a network with just one output node to this problem? (2p)**


we have to output nodes because we have 2 possible outcomes (0: not survived, and 1: survived), and we use softmax activation function to represent the probability distribution across the two classes in binary classification problems. 

The model learns to predict the likelihood of each class given the input, with each output node representing a single class. 

Nevertheless, using a single output node with a sigmoid activation function, which produces a probability between 0 and 1 indicating the likelihood that the input belongs to a specific class (usually class 1), is also feasible and effective. This single-node configuration works well for binary classification tasks and is frequently used in conjunction with the binary cross-entropy loss function.

We used softmax activation because we previously one-hot encoded the `survived` variable, transforming it into an array of 2 numbers:
- 0 = not survived
- 1 = survived

If we hadn't transformed it to an array, and left it binarized (0 or 1) as it was originally, we could have used Sigmoid activation instead, and get 1 unique output node.
 

Using Softmax activation with Categorical Crossentropy loss is only appropriate when you have two or more output nodes representing mutually exclusive classes, therefore, to adapt the network to one output node, I need to:
- Use binary `survived` class, in the original form, not in an array.
- Create a Sigmoid activation class
- Create a Binary Cross-Entropy loss class

In [74]:
X_2 = titanic_v[features].to_numpy()
Y_2 = titanic_v[target].to_numpy(dtype=int)

X_2, X_mean_2, X_std_2 = normalise(X_2)

In [75]:
X_2

array([[-1.43391396,  1.30064545, -0.05919979, -0.55163305,  3.13374271],
       [-1.43391396, -0.76811329, -2.01066414,  0.54429193,  2.06098509],
       [-1.43391396,  1.30064545, -1.93560782,  0.54429193,  2.06098509],
       ...,
       [ 0.94267619, -0.76811329, -0.23294128, -0.55163305, -0.52861549],
       [ 0.94267619, -0.76811329, -0.19819298, -0.55163305, -0.52861549],
       [ 0.94267619, -0.76811329, -0.05919979, -0.55163305, -0.51695264]],
      shape=(1045, 5))

In [76]:
Y_2

array([1, 1, 0, ..., 0, 0, 0], shape=(1045,))

In [77]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output) * self.output


class Loss_BinaryCrossentropy:
    def forward(self, y_pred, y_true):
        # Clip predictions to avoid log(0)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.output = - (y_true * np.log(y_pred_clipped) +
                         (1 - y_true) * np.log(1 - y_pred_clipped))
        return np.mean(self.output)

    def backward(self, dvalues, y_true):
    # Reshape y_true to match dvalues shape if necessary
        if len(y_true.shape) == 1:
            y_true = y_true.reshape(-1, 1)

        dvalues_clipped = np.clip(dvalues, 1e-7, 1 - 1e-7)
        self.dinputs = -(y_true / dvalues_clipped - (1 - y_true) / (1 - dvalues_clipped)) / len(dvalues)


In [78]:
np.random.seed(42) # for reproducibility
dense1 = Layer_Dense(5, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 1)
activation2 = Activation_Sigmoid()
loss_function = Loss_BinaryCrossentropy()
optimizer = Optimizer_GD(learning_rate=0.5)


for epoch in range(10001):
    dense1.forward(X_2)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    loss = loss_function.forward(activation2.output, Y_2)

    predictions = (activation2.output > 0.5).astype(int)
    accuracy = np.mean(predictions == Y_2)

    if epoch % 1000 == 0:
        print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}')

    loss_function.backward(activation2.output, Y_2)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


epoch: 0, acc: 0.576, loss: 0.693
epoch: 1000, acc: 0.533, loss: 1.081
epoch: 2000, acc: 0.533, loss: 1.100
epoch: 3000, acc: 0.533, loss: 1.120
epoch: 4000, acc: 0.533, loss: 1.124
epoch: 5000, acc: 0.533, loss: 1.126
epoch: 6000, acc: 0.533, loss: 1.128
epoch: 7000, acc: 0.533, loss: 1.130
epoch: 8000, acc: 0.533, loss: 1.130
epoch: 9000, acc: 0.533, loss: 1.131
epoch: 10000, acc: 0.533, loss: 1.129


Very bad results, and it took a very long time to run compared to the other models.

<br>


**d. Adapt the code so that it can run batches or mini batches. For an example, see solutions to Homework 3! (2p)**

Now, we are going to update the training loop so that instead of feeding all the data at once (full batch), or just one sample at a time (stochastic), we feed the data in small groups (mini-batches).

The purpose of implementing mini-batches is to:

- Speed up training, as it uses less memory than a full batch, and is faster than one-sample-at-a-time (stochastic).

- Reduces overfitting, due to some randomness added.

- Allows more stable updates than stochastic gradient descent.

In [79]:
def mini_batch(X, y, batch_size=32):
    indices = np.random.permutation(len(X))  # we shuffle data before batching
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    for start_idx in range(0, len(X), batch_size):
        end_idx = min(start_idx + batch_size, len(X))
        yield X_shuffled[start_idx:end_idx], y_shuffled[start_idx:end_idx]


np.random.seed(42)  # for reproducibility
dense1 = Layer_Dense(5, 5)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(5, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD()

batch_size = 32  

for epoch in range(10001):
    for batch_X, batch_y in mini_batch(X, training_results_array, batch_size):
        dense1.forward(batch_X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        
        loss = loss_activation.forward(dense2.output, batch_y)
        predictions = np.argmax(loss_activation.output, axis=1)
        y_train = np.argmax(batch_y, axis=1)
        
        accuracy = np.mean(predictions == y_train)
        
        if epoch % 100 == 0:
            print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}')
        
        loss_activation.backward(loss_activation.output, y_train)
        dense2.backward(loss_activation.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)


epoch: 0, acc: 0.656, loss: 0.693
epoch: 0, acc: 0.562, loss: 0.692
epoch: 0, acc: 0.812, loss: 0.687
epoch: 0, acc: 0.656, loss: 0.686
epoch: 0, acc: 0.594, loss: 0.688
epoch: 0, acc: 0.562, loss: 0.689
epoch: 0, acc: 0.500, loss: 0.694
epoch: 0, acc: 0.562, loss: 0.689
epoch: 0, acc: 0.656, loss: 0.682
epoch: 0, acc: 0.469, loss: 0.697
epoch: 0, acc: 0.469, loss: 0.697
epoch: 0, acc: 0.312, loss: 0.709
epoch: 0, acc: 0.594, loss: 0.688
epoch: 0, acc: 0.469, loss: 0.696
epoch: 0, acc: 0.469, loss: 0.696
epoch: 0, acc: 0.531, loss: 0.692
epoch: 0, acc: 0.688, loss: 0.682
epoch: 0, acc: 0.719, loss: 0.677
epoch: 0, acc: 0.500, loss: 0.694
epoch: 0, acc: 0.750, loss: 0.670
epoch: 0, acc: 0.719, loss: 0.669
epoch: 0, acc: 0.531, loss: 0.691
epoch: 0, acc: 0.469, loss: 0.700
epoch: 0, acc: 0.688, loss: 0.671
epoch: 0, acc: 0.688, loss: 0.668
epoch: 0, acc: 0.594, loss: 0.681
epoch: 0, acc: 0.594, loss: 0.681
epoch: 0, acc: 0.625, loss: 0.675
epoch: 0, acc: 0.625, loss: 0.675
epoch: 0, acc:

as we can see, we got one result per batch. Thus, 32 batches times 10000 epochs, we get 320000 results.
That’s noisy and hard to interpret, so let's just print out the averages of each epoch, for both accuracy and loss.

In [82]:
np.random.seed(42)  # for reproducibility

epoch_accuracies = []  
epoch_losses = []  

for epoch in range(10001):
    batch_accuracies = []  
    batch_losses = []  
    
    for batch_X, batch_y in mini_batch(X, training_results_array, batch_size):
        dense1.forward(batch_X)
        activation1.forward(dense1.output)
        dense2.forward(activation1.output)
        
        loss = loss_activation.forward(dense2.output, batch_y)
        predictions = np.argmax(loss_activation.output, axis=1)
        y_train = np.argmax(batch_y, axis=1)
        
        accuracy = np.mean(predictions == y_train)  
        batch_accuracies.append(accuracy)  
        
        batch_losses.append(loss)  
        
        loss_activation.backward(loss_activation.output, y_train)
        dense2.backward(loss_activation.dinputs)
        activation1.backward(dense2.dinputs)
        dense1.backward(activation1.dinputs)
        
        optimizer.update_params(dense1)
        optimizer.update_params(dense2)
    
    epoch_accuracy = np.mean(batch_accuracies)
    epoch_loss = np.mean(batch_losses)
    
    epoch_accuracies.append(epoch_accuracy)
    epoch_losses.append(epoch_loss)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch} - Average Accuracy: {epoch_accuracy:.4f}, Average Loss: {epoch_loss:.4f}")


Epoch 0 - Average Accuracy: 0.8171, Average Loss: 0.4195
Epoch 100 - Average Accuracy: 0.8204, Average Loss: 0.4223
Epoch 200 - Average Accuracy: 0.8191, Average Loss: 0.4184
Epoch 300 - Average Accuracy: 0.8200, Average Loss: 0.4178
Epoch 400 - Average Accuracy: 0.8162, Average Loss: 0.4191
Epoch 500 - Average Accuracy: 0.8200, Average Loss: 0.4188
Epoch 600 - Average Accuracy: 0.8238, Average Loss: 0.4182
Epoch 700 - Average Accuracy: 0.8185, Average Loss: 0.4206
Epoch 800 - Average Accuracy: 0.8219, Average Loss: 0.4182
Epoch 900 - Average Accuracy: 0.8195, Average Loss: 0.4203
Epoch 1000 - Average Accuracy: 0.8229, Average Loss: 0.4168
Epoch 1100 - Average Accuracy: 0.8224, Average Loss: 0.4191
Epoch 1200 - Average Accuracy: 0.8176, Average Loss: 0.4194
Epoch 1300 - Average Accuracy: 0.8215, Average Loss: 0.4176
Epoch 1400 - Average Accuracy: 0.8229, Average Loss: 0.4191
Epoch 1500 - Average Accuracy: 0.8153, Average Loss: 0.4198
Epoch 1600 - Average Accuracy: 0.8171, Average Loss:

We get very similar accuracy and loss to previous models. However, within the closeness in results to the other models, this learning method gives one of the highest accuracies, better than both the [5,5,2] and [5,10,2] models, and even lower (slightly lower) loss than the previous minimum loss achieved by [5,8,4,2] model.

Also, it is good to note that it learnt very quickly too, reaching top accuracy and low loss in the first epochs.

<br>

## Recognizing handwritten numbers 

Adapt the code in Backpropagation 1 to the task of recognizing the digits in MNIST
dataset. In the data there are n= 60,000 examples. I suggest to create a network with
[784, N, 10] layers.


In [98]:
import pickle
import numpy as np

with open('mnist.pkl', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')


inputs:

In [99]:
X, y = train_set
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(50000, 784), dtype=float32)

true outputs:

In [100]:
y

array([5, 0, 4, ..., 8, 4, 8], shape=(50000,))

**a. Why are 784 layers appropriate for the input layer? (2p)**

MNIST, which stands for Modified National Institute of Standards and Technology, is a database thant contains handwritten numbers from 0 to 9, that are frequently used to train image processing algorithms and machine learning models.


Apparently, it contains 60,000 training images and 10,000 test images, all in grayscale and sized 28x28 pixels.

Is this shape (28x28) that makes 784 layers appropriate for the input layer, as it is the number of pixels that would be entering the neural network.


<br>


**b. Why are 10 layers appropriate for the output layer? Are there other options? (3p)**

There are 10 digit classes (0 to 9, as we said), so we need 10 output neurons, one for each digit.

For the neural network, due to its multi-class classification nature, we will use the same approach as with the titanic dataset: Softmax activation + Categorical Cross-Entropy loss.

Therefore, each output neuron will represent the probability or confidence score that the input is a specific digit.

<br>


**c. Run a backpropagation algorithm on a smaller subset of the 60,000 examples and try to find reasonable value on the learning rate as well as the size of the hidden layer. (3p)**


Let's take 5000 observations

In [102]:
X_subset = X[:5000]
y_subset = y[:5000]

X_subset.shape, y_subset.shape

((5000, 784), (5000,))

In [103]:
def vectorized_result(j):
    e = np.zeros((10,))
    e[j] = 1.0
    return e

y_encoded_subset = np.array([vectorized_result(label) for label in y_subset])

y_encoded_subset.shape
y_encoded_subset[0:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

Check for standarization, as it looked that data has been already standarized.

In [ ]:
X_subset.min(), X_subset.max(), X_subset.mean(), X_subset.std()

(np.float32(0.0),
 np.float32(0.99609375),
 np.float32(0.1297137),
 np.float32(0.30633897))

yes, as we thought, pixels are already standarized to range between 0 and 1. However, originally, pixels could go from 0 to 255 in the grayscale, so we would have to divide X by 255, to standarize it.

In [ ]:
np.random.seed(42)

dense1 = Layer_Dense(784, 64)              
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 10)               
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_GD(learning_rate=0.1)


for epoch in range(1001):
    dense1.forward(X_subset)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y_encoded_subset)

    predictions = np.argmax(loss_activation.output, axis=1)
    y_labels = np.argmax(y_encoded_subset, axis=1)
    accuracy = np.mean(predictions == y_labels)

    if not epoch % 100:
        print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}')
    
    loss_activation.backward(loss_activation.output, y_labels)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


epoch: 0, acc: 0.104, loss: 2.303
epoch: 100, acc: 0.752, loss: 1.044
epoch: 200, acc: 0.877, loss: 0.496
epoch: 300, acc: 0.902, loss: 0.372
epoch: 400, acc: 0.913, loss: 0.317
epoch: 500, acc: 0.922, loss: 0.284
epoch: 600, acc: 0.929, loss: 0.260
epoch: 700, acc: 0.935, loss: 0.241
epoch: 800, acc: 0.939, loss: 0.225
epoch: 900, acc: 0.944, loss: 0.211
epoch: 1000, acc: 0.946, loss: 0.198


Wow, we get impressive results in just 1000 epochs. The accuracy of almost 95% is very high, and the loss was dramatically reduced from 2.3 to less than 0.2 (ten times lower than at the start).

The small learning rate allowed for precise parameter updates, and it didn't take much time to be such a big dataset with 784 inputs every time.

We think it is a good enough configuration for the network: lr=0.1 and 64 neurons for the hidden layer.

Why 64 neurons?:
- To be honest, in neural networks we usually try for 64, 128, 256, etc, neurons for the hidden layer, and compare.
    - We tried 64 neurons and it performed so well, that it is very likely the best configuration as it balances for both model complexity and computational efficiency. It isn't too small (which could lead to underfitting) and not too large (which could lead to overfitting and longer training times)

<br>

**d. Run the backpropagation algorithm on the entire data-set, how well is it performing?**

Without creating a subset of rows: use X and y, created at the beginning of the task.

In [105]:
y_encoded = np.array([vectorized_result(label) for label in y])

X.min(), X.max(), X.mean(), X.std()

(np.float32(0.0),
 np.float32(0.99609375),
 np.float32(0.13044983),
 np.float32(0.3072898))

Keeping same configuration of lr=0.1 and 64 neurons in the hidden layer:

In [107]:
np.random.seed(42)

dense1 = Layer_Dense(784, 64)              
activation1 = Activation_ReLU()            
dense2 = Layer_Dense(64, 10)              
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy() 
optimizer = Optimizer_GD(learning_rate=0.1)  



for epoch in range(1001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss = loss_activation.forward(dense2.output, y_encoded)

    predictions = np.argmax(loss_activation.output, axis=1)
    y_labels = np.argmax(y_encoded, axis=1)
    accuracy = np.mean(predictions == y_labels)

    if not epoch % 100:
        print(f'epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}')
    
    loss_activation.backward(loss_activation.output, y_labels)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)


epoch: 0, acc: 0.108, loss: 2.303
epoch: 100, acc: 0.749, loss: 1.100
epoch: 200, acc: 0.859, loss: 0.542
epoch: 300, acc: 0.884, loss: 0.427
epoch: 400, acc: 0.894, loss: 0.378
epoch: 500, acc: 0.901, loss: 0.350
epoch: 600, acc: 0.906, loss: 0.330
epoch: 700, acc: 0.911, loss: 0.315
epoch: 800, acc: 0.914, loss: 0.302
epoch: 900, acc: 0.917, loss: 0.291
epoch: 1000, acc: 0.920, loss: 0.281


We get very similar results. It underperforms the previous model with a sample of 5000 observations, and it took a much longer time to run, but it was expected as the whole datset is 10 times larger.

In conclusion, when trained on a smaller subset of 5000 samples as opposed to the entire MNIST dataset, the model learns more quickly and reaches higher accuracy. Training on the complete dataset improves the model's generalization and makes it more robust to the complexity and diversity of the entire dataset, even though it takes longer and yields a slightly lower accuracy (92%). 

However, the accuracy difference is just showing that the full dataset improves generalization while the subset allows for faster convergence. We think, both models exhibit impressive performance by the end of training.